<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Isidora Luck
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/IsidoraLuck/MDS7202.git`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/online_retail_II_cleaned.pickle'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle(path)
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head(10)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
def custom_features(dataframe_in):
    # Copiamos el DataFrame
    df = dataframe_in.copy()
    length_df = df.groupby("Customer ID")["InvoiceDate"].agg(lambda x: (x.max() - x.min()).days)
    recency_df = df.groupby("Customer ID")["InvoiceDate"].agg(lambda x: (pd.to_datetime("2023-10-06") - x.max()).days)
    df["TotalAmount"] = df["Quantity"] * df["Price"]
    monetary_df = df.groupby("Customer ID")["TotalAmount"].mean().round(2)
    frequency_df = df.groupby("Customer ID")["Invoice"].count()
    df["IVT"] = df.groupby("Customer ID")["InvoiceDate"].diff().dt.days
    periodicity_df = df.groupby("Customer ID")["IVT"].std().round(2)
    periodicity_df = periodicity_df.fillna(0) #reemplazar nan con 0
    # Crear un nuevo DataFrame con las características LRMFP
    lrmfp_df = pd.DataFrame({
        "Customer ID": length_df.index,
        "Length": length_df.values,
        "Recency": recency_df.values,
        "Frequency": frequency_df.values,
        "Monetary": monetary_df.values,
        "Periodicity": periodicity_df.values
    })

    return lrmfp_df


In [5]:
display(custom_features(df_retail))

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,4847,33,11.30,21.72
1,12347.0,37,4685,71,18.64,4.42
2,12348.0,0,4756,20,11.11,0.00
3,12349.0,181,4725,102,26.19,16.20
4,12351.0,0,4693,21,14.33,0.00
...,...,...,...,...,...,...
4309,18283.0,275,4700,217,2.85,11.78
4310,18284.0,0,4749,28,16.49,0.00
4311,18285.0,0,4978,12,35.58,0.00
4312,18286.0,247,4794,67,19.35,30.40


In [6]:
custom_features(df_retail).isna().sum()

Customer ID    0
Length         0
Recency        0
Frequency      0
Monetary       0
Periodicity    0
dtype: int64

**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # Si no se especifican columnas, aplicar Min-Max a todas las columnas numéricas.
        if self.columns is None:
            self.columns = X.select_dtypes(include=[pd.np.number]).columns

        X_transformed = X.copy()

        for column in self.columns:
            min_value = X[column].min()
            max_value = X[column].max()
            X_transformed[column] = (X[column] - min_value) / (max_value - min_value)

        return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [8]:
paso1 = ('custom_features', FunctionTransformer(func=custom_features))
paso2 = ('min_max_scaler', ColumnTransformer(transformers=[('min_max', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity'])]))
paso3 = ('tsne', TSNE(n_components=2, random_state=20846340))

# Configurar el pipeline
pasos = [paso1, paso2, paso3]
pipeline = Pipeline(pasos)

# Aplicar el pipeline a los datos LRMFP
X_tsne = pipeline.fit_transform(df_retail)

# Crear un DataFrame con las componentes t-SNE
tsne_df = pd.DataFrame(data=X_tsne, columns=['Component 1', 'Component 2'])

# Gráfica de las componentes t-SNE con Plotly
fig = px.scatter(tsne_df, x='Component 1', y='Component 2', title='t-SNE Components')
fig.show()


<ipython-input-7-d91cbc03cb06>:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  self.columns = X.select_dtypes(include=[pd.np.number]).columns


### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:** Se escogen 3 clusters pues desde allí empieza a aplanarse la ganancia de identificar a distintos clusters, los otros clusters extras dejan de ser tan representativos en cuanto a cantidad de datos que tienen para el valor informativo que aportan.

In [9]:
import warnings

# Desactivar todas las advertencias
warnings.filterwarnings("ignore")

In [10]:
# Definir el rango de valores de k
k_values = range(1, 21)  # k en [1, 20]

# Crear una lista para almacenar las inercias (sumas de cuadrados intra-cluster) para cada valor de k
inertias = []

# Iterar sobre los diferentes valores de k
for k in k_values:
    # Configurar el pipeline para el K-Means
    paso1 = ('custom_features', FunctionTransformer(func=custom_features))
    paso2 = ('min_max_scaler', ColumnTransformer(transformers=[('min_max', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity'])]))
    paso3 = ('kmeans', KMeans(n_clusters=k, random_state=20846340))

    pasos = [paso1, paso2, paso3]
    pipeline = Pipeline(pasos)

    # Aplicar el pipeline a los datos LRMFP
    X_kmeans = pipeline.fit_transform(df_retail)

    # Calcular la inercia y guardarla en la lista
    inertias.append(pipeline.named_steps['kmeans'].inertia_)

# Graficar la curva de codo
fig = px.line(x=k_values, y=inertias, title='Método del Codo para K-Means')
fig.update_xaxes(title='Número de Clusters (k)')
fig.update_yaxes(title='Inercia')
fig.show()


#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:** Es posible encontrar logica en las 3 agrupaciones generadas. Hay clientes que son más frecuentes su monto no es destacable, son clientes más fieles pues su frecuencia y lenght es mayor, por otra parte se tienen clientes que no son tan frecuentes pero transan por montos mayores, son importantes por el ticket de compra que generan. Finalmente esta el cliente más esporadico, no compra muy seguido ni compra montos altos, es el que no siente fidelidad con la empresa.

In [11]:
# Elegir el valor de k
k_chosen = 3  # Utiliza el valor de k que seleccionaste en la sección anterior

# Configurar el pipeline para K-Means con el valor de k elegido
paso1 = ('custom_features', FunctionTransformer(func=custom_features))
paso2 = ('min_max_scaler', ColumnTransformer(transformers=[('min_max', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity'])]))
paso3 = ('kmeans', KMeans(n_clusters=k_chosen, random_state=20846340))

pasos = [paso1, paso2, paso3]
pipeline = Pipeline(pasos)

# Aplicar el pipeline al DataFrame original
X_kmeans = pipeline.fit_transform(df_retail)  # Asegúrate de que se utilice df_retail aquí

# Obtener las etiquetas de clúster
labels = pipeline.named_steps['kmeans'].labels_

# Crear un DataFrame con las etiquetas de clúster
cluster_df = pd.DataFrame({'Cluster': labels})

# Unir el DataFrame de etiquetas de clúster con el DataFrame original
if 'Cluster' not in df_retail.columns:
    df_retail_2 = pd.concat([custom_features(df_retail), cluster_df], axis=1)

# Calcular los promedios por clúster
cluster_means = df_retail_2.groupby('Cluster').agg({'Length': 'mean', 'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean', 'Periodicity': 'mean'}).reset_index().round(1)

# Imprimir la tabla con los promedios
display(cluster_means)



,Cluster,Length,Recency,Frequency,Monetary,Periodicity
0,0,39.2,4736.6,48.5,32.3,5.2
1,1,277.2,4719.3,167.0,32.4,21.1
2,2,23.6,4933.6,28.5,53.3,3.6


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:** Hay sectores donde los cluster no se separan bien, esto puede deberse en parte al reemplazo de los valores faltantes por 0, sin embargo aún así los clusters no tienen separaciones notorias, quizas habria que usar otro metodo de clusterización.

In [12]:
if 'Cluster' not in tsne_df.columns:
   tsne_df_2 = pd.concat([tsne_df, cluster_df], axis=1)

In [13]:
# Asumiendo que tsne_df tiene las columnas 'tsne_x', 'tsne_y' y 'Cluster'
fig = px.scatter(tsne_df_2, x='Component 1', y='Component 2', color='Cluster',
                 color_continuous_scale=px.colors.sequential.Plasma,
                 title='T-SNE con Clusters KMeans')

fig.show()



# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>